In [1]:
import os
import subprocess
import uuid
import IPython.display as ipd
from playlist_copilot.utils.smp_utils import get_track_info

In [ ]:
# Function to read .m3u file and extract MP3 file paths
def read_m3u(file_path):
    mp3_file_paths = []

    with open(file_path, 'r') as file:
        for line in file:
            # Skip empty lines and lines that start with '#'
            if line.strip() and not line.startswith('#'):
                mp3_file_paths.append(line.strip())

    return mp3_file_paths

# Function to create and display an audio player for each MP3 file
def play_mp3_files(file_paths):
    for path in file_paths:
        track_id = os.path.splitext(os.path.basename(path))[0]
        track_info = get_track_info(f'spotify:track:{track_id}')
        artist = track_info['artist']
        title = track_info['title']
        album = track_info['album']
        print(f"{artist} | {title} | {album}")
        display(ipd.Audio(path))

# Function to display the songs selected
def print_playlist_songnames(file_paths):
    n = 1
    for path in file_paths:
        track_id = os.path.splitext(os.path.basename(path))[0]
        track_info = get_track_info(f'spotify:track:{track_id}')
        artist = track_info['artist']
        title = track_info['title']
        album = track_info['album']
        print(f"{n}. {artist} | {title} | {album}")
        n += 1

In [ ]:
# Song paths for ease of working

# ZAYN - Still Got Time
song_1 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/000xQL6tZNLJzIrtIgxqSl.mp3'
# Sean Turk - Middle
song_2 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/0032P9X3AD4AQvN7yzpmTo.mp3'
# 
song_3 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/004XT7kCZUEJkVIZjmBdDi.mp3'
#
song_4 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/004skCQeDn1iLntSom0rRr.mp3'
#
song_5 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/005X0FmdtkM1kiutosXLTR.mp3'
#
song_6 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/007ogFejDqJKzEXDUDF8Nf.mp3'
#
song_7 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/00AvOVhsUi1gOCnHxTFw7i.mp3'
#
song_8 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/00AxNl4D4jHL2AEf1W55j5.mp3'
#
song_9 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/00BTDWR6xCaGdst6cPKkpF.mp3'
#
song_10 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/audio/00Ci0EXS4fNPnkTbS6wkOh.mp3'

In [ ]:
# From test_subset

# Con Bro Chill - Dance Thief
test_1 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/test_subset/00AVE5t5MUtyGJf51V1GJG.mp3'

# Fantasia - Ain't Gon' Beg You
test_2 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/test_subset/02wVmJecRUCtkppST87OLx.mp3'

# swim good now - Since U Asked
test_3 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/test_subset/0Axtkw6bMF4V4HMoKd5C2g.mp3'

# Halsey - Tokyo Narita Freestyle
test_4 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/test_subset/0W182mlOojwvO1HoXPkHXY.mp3'

# Original Broadway Cast Orchestra - I'll Never Be Jealous Again [The Pajama Game]
test_5 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/test_subset/0qCT5x6NI9wSea0weWPjbI.mp3'

# Fatboy Slim - Praise You Radio Edit
test_6 = '/home/aditi/Code/tech/playlist-copilot/data/spotify-million-playlists/test_subset/2tn9zXqIZJbgL1swqwOktz.mp3'


In [ ]:
# USE CASE 1:= Starting from a single seed song

# Choose parameters and song
input_file = test_5

# Additional parameters, if needed
num_songs = '10'  # Number of songs in the playlist
noise = '0'   # Amount of noise in the playlist (default 0)
lookback = '3' # Amount of lookback in the playlist (default 3)

# Path to Python script
script = 'Deej-A.I.py'

# Arguments for the script
mp3tovec = 'mp3tovec'
output_file = str(uuid.uuid4()) + '.m3u'

# Construct the command as a list
command = ['python', script, 'Pickles_test', mp3tovec, '--playlist', output_file,
           '--nsongs', num_songs, '--noise', noise, '--lookback', lookback,
           '--inputsong', input_file]

# Execute the command
subprocess.run(command)

m3u_file_path = output_file

# Read the .m3u file and get the MP3 file paths
mp3_file_paths = read_m3u(m3u_file_path)
print_playlist_songnames(mp3_file_paths)

In [ ]:
# Use this cell to listen to the audios, if needed

# Call the function with the placeholder paths
play_mp3_files(mp3_file_paths)